### 1 介绍

消息是 LangChain 中模型上下文的基本单元。它们代表模型的输入和输出，携带在与 LLM 交互时表示对话状态所需的内容和元数据。

消息是包含以下内容的对象

role - 标识消息类型（例如 system ， user ）

content - 表示消息的实际内容（如文本、图像、音频、文档等）

Metadata - 可选字段，如响应信息、消息 ID 和令牌使用情况

### 2 基本用法

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("deepseek-chat")

system_msg = SystemMessage("你是一个优秀的AI助手.")
human_msg = HumanMessage("请你介绍下自己")

messages = [system_msg, human_msg]
response = model.invoke(messages)  
response

### 3 消息类型

System message 告诉模型如何表现并为交互提供上下文

Human message 表示用户输入和与模型的交互

AI message 由模型生成的响应，包括文本内容、工具调用和元数据

Tool message 表示工具调用的输出

#### 3.1 System Message

In [ ]:
from langchain.messages import SystemMessage, HumanMessage
# system_msg = SystemMessage("你是一个优秀的AI助手")  简略版
system_msg = SystemMessage("""
您是一名资深的人工智能专家，专长于人工智能领域的研究。请始终提供通俗易懂的自然语言来描述复杂的人工智能概念。在描述中做到简洁但详尽。
""")

messages = [
    system_msg,
    HumanMessage("强化学习和深度学习的区别")
]
response = model.invoke(messages)
response

#### 3.2 Human Message

In [ ]:
# response = model.invoke([
#   HumanMessage("什么是机器学习？")
# ])  简单使用

# 带元数据的human message
human_msg = HumanMessage(
    content="Hello!",
    # name 字段的行为因模型而异——有些用于用户识别，有些则忽略。
    name="alice",  # 可选：识别不同的用户
    id="msg_123",  # 可选：用于追踪的唯一标识符
)

#### 3.3 AI Message

 代表模型调用的输出。它们可以包括多模态数据、工具调用以及提供者特定的元数据，您可以在后续访问。

调用模型时返回的 AIMessage 对象，包含所有相关的元数据。

提供者会根据类型对消息进行不同的权重/上下文化，这意味着有时手动创建一个新的 AIMessage 对象并将其插入到消息历史中，仿佛它是由模型生成的，会更有帮助。

In [3]:
from langchain.messages import AIMessage, SystemMessage, HumanMessage

# 手动创建AI消息（例如，用于对话历史）
ai_msg = AIMessage("I'd be happy to help you with that question!")

# 添加到对话历史
messages = [
    SystemMessage("You are a helpful assistant"),
    HumanMessage("Can you help me?"),
    ai_msg,  # Insert as if it came from the model
    HumanMessage("Great! What's 2+2?")
]

response = model.invoke(messages)
response

AIMessage(content='2 + 2 = 4', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 38, 'total_tokens': 45, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 38}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '50b9ceee-aaa2-48c5-9a70-d45cecf399fb', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--a3285564-f79d-4d81-9ece-b54bd55c67f1-0', usage_metadata={'input_tokens': 38, 'output_tokens': 7, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [ ]:
print(response.text) # 消息的文本内容。  只能输出text文本类型
print(response.content) # 消息的原始内容。 因为content支持接收多模态内容，可以输出多种类型。
print(response.content_blocks) # 内容块
print(response.tool_calls) # 工具调用
print(response.id) # id
print(response.usage_metadata) # 消息的使用元数据，可能包含可用的令牌数。
print(response.response_metadata) # 消息的响应元数据

2 + 2 = 4
2 + 2 = 4
[{'type': 'text', 'text': '2 + 2 = 4'}]
[]
lc_run--a3285564-f79d-4d81-9ece-b54bd55c67f1-0
{'input_tokens': 38, 'output_tokens': 7, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
{'token_usage': {'completion_tokens': 7, 'prompt_tokens': 38, 'total_tokens': 45, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 38}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '50b9ceee-aaa2-48c5-9a70-d45cecf399fb', 'finish_reason': 'stop', 'logprobs': None}


In [ ]:
chunks = []
full_message = None
for chunk in model.stream("Hi,请你详细介绍下自己"):
    chunks.append(chunk)
    print(chunk.text, end='', flush=True)
    full_message = chunk if full_message is None else full_message + chunk

你好！很高兴认识你！😊

我是DeepSeek，由深度求索公司创造的AI助手。让我详细介绍一下自己：

## 基本信息
- **身份**：DeepSeek最新版本模型
- **创造者**：深度求索公司
- **知识截止**：2024年7月
- **上下文长度**：128K

## 我的能力
- **文本处理**：可以帮你写作、翻译、分析、总结等各种文本任务
- **文件处理**：支持上传图像、txt、pdf、ppt、word、excel文件，并从中读取文字信息进行处理
- **联网搜索**：支持实时联网搜索功能（需要你在Web/App中手动开启）
- **多领域知识**：涵盖科学、技术、文学、历史等各个领域

## 我的特点
- **完全免费**：目前没有任何收费计划，你可以放心使用
- **热情细腻**：我喜欢用温暖、细致的方式与你交流
- **逻辑清晰**：善于分析复杂问题，提供结构化思考

## 使用方式
你可以通过官方应用商店下载DeepSeek App，或者直接在网页端使用。

我会尽我所能帮助你解答问题、协助工作学习，或者只是陪你聊天！有什么想了解的或需要帮助的，尽管告诉我吧！✨

#### 3.4 Tool Message

In [16]:
from langchain.tools import tool

@tool
def get_weather(city):
    """查询city的天气"""
    return f"{city}的天气是晴天"

model_with_tool = model.bind_tools([get_weather])
ai_result = model_with_tool.invoke("请帮我查询下北京的天气")
ai_result

AIMessage(content='我来帮您查询北京的天气情况。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 146, 'total_tokens': 174, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 18}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'afa3da4b-d4a3-4d26-b636-4a404bf47523', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f577f2a0-e7d3-4142-963f-a96aa817b2c1-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '北京'}, 'id': 'call_00_Q12Ebnx2oZYAmvT88BvlSt0y', 'type': 'tool_call'}], usage_metadata={'input_tokens': 146, 'output_tokens': 28, 'total_tokens': 174, 'input_token_details': {'cache_read': 128}, 'output_token_details': {}})

In [18]:
tool_result = get_weather.invoke(ai_result.tool_calls[0])
tool_result

ToolMessage(content='北京的天气是晴天', name='get_weather', tool_call_id='call_00_Q12Ebnx2oZYAmvT88BvlSt0y')

In [ ]:
# content 工具调用的字符串化输出。
# tool_call_id 此消息响应的工具调用的 ID。（必须与 AIMessage 中的工具调用 ID 匹配）
# name 被调用的工具的名称。
# artifact 未发送到模型但可以通过编程访问的附加数据。
# artifact 字段存储补充数据，这些数据不会发送给模型，但可以通过程序访问。这对于存储原始结果、调试信息或下游处理的数据非常有用，而不会影响模型的上下文。

### 4 消息内容

你可以将消息的内容视为发送给模型的数据负载。消息具有一个 content 属性，该属性类型较为宽松，支持字符串和未类型化对象的列表（例如字典）。这允许在 LangChain 聊天模型中直接支持提供者原生结构，如多模态内容和其他数据。

另外，LangChain 提供了专用的内容类型，用于文本、推理、引用、多模态数据、服务器端工具调用以及其他消息内容。

In [ ]:
# 多模态示例
from langchain.messages import HumanMessage

# String content
human_message = HumanMessage("Hello, how are you?")

# Provider-native format (e.g., OpenAI)
human_message = HumanMessage(content=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
])

# List of standard content blocks
human_message = HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"},
])

In [ ]:
# 演示content_blocks块的内容。具体用到了看官网
from langchain.messages import AIMessage

message = AIMessage(
    content=[
        {
            "type": "reasoning",
            "id": "rs_abc123",
            "summary": [
                {"type": "summary_text", "text": "summary 1"},
                {"type": "summary_text", "text": "summary 2"},
            ],
        },
        {"type": "text", "text": "...", "id": "msg_abc123"},
    ],
    response_metadata={"model_provider": "openai"}
)

message.content_blocks

[{'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 1'},
 {'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 2'},
 {'type': 'text', 'text': '...', 'id': 'msg_abc123'}]